In [21]:
# importing packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.gridspec import GridSpec

#%matplotlib qt

from scipy.optimize import curve_fit
from scipy.special import expit

import ssm
from ssm import transitions 

import scipy.io
import os
import seaborn as sns
import pandas as pd

In [22]:
# load data
filename = 'C:/Users/Asus/Desktop/PhD/rotations/AnnJeff/data/riskdata/muscimoldata.mat'
#filename = '/Users/mpexirra/Documents/data/controldata'
data = scipy.io.loadmat(filename)
data = data['muscimoldata']
df = pd.DataFrame(data)
df.columns = ['animal', 'sessionid', 'trialnumber', 'trial_block', 'lotterymag', 'lotteryprob', 
           'surebetmag', 'rewardreceived', 'lotterychoice', 'lotteryoutcome', 'musdose', 'region']

In [23]:
# defining plot colors
stimcolors = sns.color_palette("rocket_r")
colors = ['gold', 'plum', 'lightskyblue']

In [24]:
# getting relevant variables from data
animals = df.animal.unique()
nanimals = len(animals)

# creating extra variables in the dataframe
df['choiceout'] = np.zeros(len(df))
df.loc[(df.lotterychoice == 1) & (df.lotteryoutcome == 1),['choiceout']] = 1
df.loc[(df.lotterychoice == 1) & (df.lotteryoutcome == 0),['choiceout']] = -1

df.loc[df.region == 1, 'region'] = 'ctrl'
df.loc[df.region == 2, 'region'] = 'fof'
df.loc[df.region == 3, 'region'] = 'ppc'

dftraining = df[df.region == 'ctrl']

In [25]:
# normalization function
def normV(X, method):   
    if method == 'max':
        return X/max(X)
    if method == 'zscore':
        return (X - np.mean(X))/np.std(X)

In [26]:
# creating input structure compatible with the ssm package
inputdim = 2
inputs = []
Y = []

for aa in range(nanimals):
    animaldata = dftraining[dftraining.animal == animals[aa]]
    sessions = animaldata.sessionid.unique()
    nsessions = len(sessions)
    
    inputsaux = []
    yaux = []
    for ss in range(nsessions):
        sessiondata = animaldata[animaldata.sessionid == sessions[ss]]
        ntrials = len(sessiondata)
        inputsaux.append(np.ones([ntrials, inputdim]))
        yaux.append(np.zeros([ntrials,1], dtype = int))
        
        sbmag = sessiondata.surebetmag[sessiondata.lotterychoice == 0].unique()
        sbrwd = sessiondata.rewardreceived[sessiondata.lotterychoice == 0].unique()
        rwdmult = sbrwd/sbmag
        lotteryprob = sessiondata.lotteryprob.unique()
        
        # getting and saving deltaEV information
        deltaEV = rwdmult * (sessiondata.lotterymag * lotteryprob) - sbrwd
        normdeltaEV = normV(deltaEV, 'max')
        inputsaux[ss][:,0] = normdeltaEV
        yaux[ss][:,0] = sessiondata.lotterychoice
        
        ## getting and saving previous choice and previous reward information
        #inputsaux[ss][:,2] = np.hstack((0, sessiondata.lotterychoice[1:]))
        #inputsaux[ss][:,2] = np.hstack((0, sessiondata.choiceout[1:]))
    
    inputs.append(inputsaux)
    Y.append(yaux)

In [27]:
# setting glm-hmm parameters
nstates = 3
obsdim = 1
ncatergories = 2
niterations = 200
# creating new df variables for estimated states from glmhmm
dftraining['pstate1'] = np.zeros(len(dftraining))
dftraining['pstate2'] = np.zeros(len(dftraining))
dftraining['pstate3'] = np.zeros(len(dftraining))
dftraining['state'] = np.zeros(len(dftraining))

# creating variables to keep from HMM
w = []
transmat = []

for aa in range(nanimals):
    w.append(np.zeros([nstates, inputdim]))
    transmat.append(np.zeros([nstates, nstates]))
    
    glmhmm = ssm.HMM(nstates, obsdim, inputdim,  observations = 'input_driven_obs',
                observation_kwargs = dict(C = ncatergories), transitions = "standard")
    glmhmm2fit = glmhmm.fit(Y[aa], inputs = inputs[aa], method = 'em', num_iters = niterations, tolerance = 10**-5)

    # saving the glm weights and transition matrix values
    waux = - glmhmm.observations.params.reshape(nstates, inputdim)
    w[aa] = waux

    transmat[aa] = glmhmm.transitions.transition_matrix
    
    # getting expected states
    posterior_pstate = [glmhmm.expected_states(data = data, input = inputs)[0]
                        for data, inputs
                        in zip(Y[aa], inputs[aa])]
   
    # saving expected states  for each trialin the data frame
    ppstate_aux = np.array([item for sublist in posterior_pstate for item in sublist])
    dftraining.loc[dftraining.animal == animals[aa],['pstate1', 'pstate2','pstate3']] = ppstate_aux

dftraining.state = dftraining[['pstate1', 'pstate2', 'pstate3']].idxmax(axis = 1)    

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

  0%|          | 0/200 [00:00<?, ?it/s]

C:\Users\Asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

C:\Users\Asus\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [38]:
# getting manipulation sessions expected states based on the control data model

array([2.61753288e-01, 2.37014000e-09, 7.38246709e-01])